#### Import des bibliothèqes :

In [ ]:
import random
from math import ceil, gcd
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import kurtosis, skew
from statsmodels.graphics.gofplots import qqplot_2samples

#### Fonctions utilitaires :

In [ ]:

def get_seed() -> int:
    return int(random.random() * 1_000_000)


def linear_congruential_generator(m: int, a: int, c: int, seed: int, n: int) -> List[int]:
    res = [seed]
    for i in range(n):
        seed = (a * seed + c) % m
        res.append(seed)
    return res

# I - Présentation des générateurs

## a - Park & Miller / Standard Minimal

Le générateur de nombres pseudo-aléatoires Park & Miller est un générateur standard convenablement testé et portable.
Il utilise le Générateur Congruentiel Linéraire avec les paramètres suivants :
- m = 2^31 - 1,
- a = 16807,
- c = 0.

Avec ce type de générateur, chaque valeur générée dépend de la valeur précédente.

In [ ]:
def park_miller(seed: int, n: int) -> List[int]:
    if seed == 0:
        raise ValueError("Park-Miller should not start with a seed equal to 0.")
    m = pow(2, 31) - 1
    return linear_congruential_generator(m, 16807, 0, seed, n)


def park_miller_normalized(seed: int, n: int) -> List[float]:
    return [elem / (pow(2, 31) - 1) for elem in park_miller(seed, n)]

## b - Mitchell & Moore

Le générateur de nombres pseudo-aléatoires aléatoires Mitchell & Moore utilise une suite de Fibonacci "retardée",
avec les paramètres suivants :

- k = 55
- l = 24

Ces paramètres nécessitent d'initialiser le générateur avec 56 nombres pseudo-aléatoires.
Pour ce faire, j'ai choisi d'utilisé le générateur Parker & Miller vu ci-dessus.

Les générateurs basés sur des suites de Fibonacci "retardées" produisent des résultats bien supérieurs
à ceux basés sur des congruences linéaires, grâce au "décalage" introduit qui évite de générer une valeur aléatoire
liée à la précédente.

In [ ]:
def mitchell_moore(seed: int, n: int, m: int) -> List[float]:
    res = park_miller(seed, 56)

    for i in range(57, 57 + n):
        res.append((res[i - 24] + res[i - 55]) % m)

    return [e / m for e in res[57:]]  # normalisation

## c - Mersenne Twister

Le générateur de nombres pseudo-aléatoires aléatoires Mersenne Twister est basé sur un TGSFR (Twisted Generalised Shift Feedback Register, un type particulier de registre à décalage à rétroaction).

Ce générateur a été optimisé pour être utilisé dans le cadre de simulations de Monte-Carlo dans un grand nombre de domaines scientifiques.Il est le générateur de nombres pseudo-aléatoires par défaut en Python, Ruby, R, PHP, MATLAB, Stata et C++.


In [ ]:
def mersenne_twister(seed: int, n: int) -> List[float]:
    random.seed(seed)
    return [random.random() for _ in range(n)]

## d - Blum Blum Shub

Le générateur de nombres pseudo-aléatoires aléatoires Blum Blum Shub n'est pas approprié aux simulations, mais plutôt à la cryptographie, car il est assez lent.

In [ ]:
def blum_blum_shub(seed: int, n: int) -> List[float]:  # intéret --> difficile à hacker
    m = 56923661
    if seed == 0 or seed == 1:
        raise ValueError("Seed should not be equal to 0 or 1.")
    if gcd(m, seed) != 1:
        raise ValueError(f"Seed should be prime with modulo : {m}")

    res = [seed]
    for i in range(n):
        seed = (seed * seed) % m
        res.append(seed)
    return [e / m for e in res]  # normalisation

# II - Test des générateurs

## a - Interprétation graphique

In [ ]:
GENERATORS_NAMES = ['Park & Miller', 'Mitchell & Moore', 'Blum Blum Shub', 'Mersenne Twister']


def generate(n: int) -> List[List[float]]:
    return [park_miller_normalized(get_seed(), n),
            mitchell_moore(get_seed(), n, 1_000_000),
            blum_blum_shub(get_seed(), n),
            mersenne_twister(get_seed(), n)]


def scatter_plot(n: int) -> None:
    l1 = generate(n)
    l2 = generate(n)
    title = f'Nuage de points montrant la répartition des nombres générés pour n = {n}.'

    fig, ax = plt.subplots(nrows=2, ncols=2)
    fig.suptitle(title, fontsize=20, y=1.02)
    fig.set_size_inches(15, 10)
    i = 0
    for row in ax:
        for col in row:
            col.scatter(l1[i], l2[i], alpha=0.5, s=1)
            col.title.set_text(GENERATORS_NAMES[i])
            i += 1
    fig.tight_layout(pad=3.0)
    plt.show()


scatter_plot(100)
scatter_plot(10_000)
scatter_plot(1_000_000)

In [ ]:
def histo_plot(n: int) -> None:
    l = generate(n)
    title = f'Histogramme montrant la répartition des nombres générés pour n = {n}.'

    fig, ax = plt.subplots(nrows=2, ncols=2)
    fig.suptitle(title, fontsize=20, y=1.02)
    fig.set_size_inches(15, 10)
    i = 0
    for row in ax:
        for col in row:
            col.hist(l[i])
            col.title.set_text(GENERATORS_NAMES[i])
            i += 1
    fig.tight_layout(pad=3.0)
    plt.show()


histo_plot(100)
histo_plot(10_000)
histo_plot(1_000_000)

In [ ]:
def qq_plot(n: int) -> None:
    l1 = generate(n)
    l2 = generate(n)
    title = f'QQplot montrant la répartition des nombres générés pour n = {n}.'

    fig, ax = plt.subplots(nrows=2, ncols=2)
    fig.suptitle(title, fontsize=20, y=1.02)
    fig.set_size_inches(15, 10)
    i = 0
    for row in ax:
        for col in row:
            pp_x = sm.ProbPlot(np.array(l1[i]))
            pp_y = sm.ProbPlot(np.array(l2[i]))
            qqplot_2samples(pp_x, pp_y, line="45", ax=col)
            col.title.set_text(GENERATORS_NAMES[i])
            i += 1
    fig.tight_layout(pad=3.0)
    plt.show()


qq_plot(100)
qq_plot(10_000)
qq_plot(1_000_000)

## b - Tests Mathématiques

### Equiprobabilité

In [ ]:
def get_percentile(values: List[float], percentage: float) -> float:
    sorted_list = sorted(values)
    return sorted_list[ceil(len(values) * (percentage / 100))]


n = 1_000_000

l = [park_miller_normalized(get_seed(), n),
     mitchell_moore(get_seed(), n, 1_000_000),
     blum_blum_shub(get_seed(), n),
     mersenne_twister(get_seed(), n)]

data = {'Quartile 25%': [get_percentile(sublist, 25) for sublist in l],
        'Mean value': [get_percentile(sublist, 50) for sublist in l],
        'Quartile 75%': [get_percentile(sublist, 75) for sublist in l],
        'Variance': [np.var(sublist) for sublist in l],
        'Kurtosis (normalised)': [kurtosis(sublist) for sublist in l],
        'Skewness': [skew(sublist) for sublist in l]}

pd.DataFrame.from_dict(data, orient='index', columns=GENERATORS_NAMES)

### Tests quantitatifs usuels